In [1]:
import os
import pandas as pd
import numpy as np
from convert_eprime import convert as ep

In [2]:
source_dir = os.path.join('..','sourcedata')
derivs_dir = os.path.join('..','derivatives')

# Convert all N-back data to csv

#### Define a function to clean the N-back data

Stack the blocks vertically instead of horizontally, label the trial rows properly, and tag each trial as a HIT, MISS, FA, CR. We are also ouputting a new CSV data file in the sourcedata folder, all cleaned-up.

In [3]:
def nstack_score_label(fpath,outpath):
    df = pd.read_excel(fpath)
    
    # Hierarchicalize the column index
    df.columns=pd.MultiIndex.from_tuples([
        (df.columns[0].split('.')[0],df.columns[0].split('.')[1]),
        (df.columns[1].split('.')[0],df.columns[1].split('.')[1]),
        (df.columns[2].split('.')[0],df.columns[2].split('.')[1]),
        (df.columns[3].split('.')[0],df.columns[3].split('.')[1]),
        (df.columns[4].split('.')[0],df.columns[4].split('.')[1]),
        (df.columns[5].split('.')[0],df.columns[5].split('.')[1]),
    ])
    
    # Stack blocks, Reset trial row index, and Rename columns to be descriptive
    df = df.stack(0).reset_index().rename(
        columns={'level_0':'trial','level_1':'block'}
    ).sort_values(['block','trial'])
    df['sub'] = os.path.basename(fpath).split('_')[0].split('-')[1]
    df['block'] = df['block'].str[1]
    df['trial'] = df['trial'] + 1
    df = df.set_index(['sub','block','trial'])
    
    # Determine Hits, CRs, FAs
    cr_mask = (df['Rsp'] == 0) & (df['CRsp'] == 0)
    ms_mask = (df['Rsp'] == 0) & (df['CRsp'] == 1)
    fa_mask = (df['Rsp'] == 1) & (df['CRsp'] == 0)
    ht_mask = (df['Rsp'] == 1) & (df['CRsp'] == 1)
    df['CR']   = cr_mask.astype(int)
    df['MISS'] = ms_mask.astype(int)
    df['FA']   = fa_mask.astype(int)
    df['HIT']  = ht_mask.astype(int)
    
    # Convert RT 0 to RT NaN
    df['RT'] = df['RT'].replace(0,np.NaN)
    
    # Output to new CSV datafile
    df.to_csv(outpath)
    print('Output file successfully created- ',outpath)

#### Read all the subject data

Executing N-back data cleaning & Setting up for subject-level analysis.

In [7]:
for s in os.listdir(source_dir):
    if s.startswith('sub-4'):
        sub_dir = os.path.join(source_dir,s)
        for f in os.listdir(sub_dir):
            if f.endswith('.xlsx') and f.split('_')[-2] == 'task-nback':
                #print(f)
                o = f.split('.')[0]+'.csv'
                fpath = os.path.join(sub_dir,f)
                opath = os.path.join(sub_dir,o)
                if os.path.isfile(opath):
                    print(opath,'exists')
                else:
                    o = os.path.basename(fpath).split('.')[0]+'.csv' 
                    opath = os.path.join(sub_dir,o)
                    print(fpath,'=>',opath)
                    nstack_score_label(fpath,opath)            

..\sourcedata\sub-401\sub-401_task-nback_beh.csv exists
..\sourcedata\sub-402\sub-402_task-nback_beh.csv exists
..\sourcedata\sub-403\sub-403_task-nback_beh.csv exists
..\sourcedata\sub-404\sub-404_task-nback_beh.csv exists
..\sourcedata\sub-405\sub-405_task-nback_beh.csv exists
..\sourcedata\sub-406\sub-406_task-nback_beh.csv exists
..\sourcedata\sub-407\sub-407_task-nback_beh.csv exists
..\sourcedata\sub-408\sub-408_task-nback_beh.csv exists
..\sourcedata\sub-409\sub-409_task-nback_beh.csv exists
..\sourcedata\sub-410\sub-410_task-nback_beh.csv exists
..\sourcedata\sub-411\sub-411_task-nback_beh.csv exists
..\sourcedata\sub-412\sub-412_task-nback_beh.csv exists
..\sourcedata\sub-413\sub-413_task-nback_beh.csv exists
..\sourcedata\sub-414\sub-414_task-nback_beh.csv exists
..\sourcedata\sub-415\sub-415_task-nback_beh.csv exists
..\sourcedata\sub-416\sub-416_task-nback_beh.csv exists
..\sourcedata\sub-419\sub-419_task-nback_beh.csv exists
..\sourcedata\sub-420\sub-420_task-nback_beh.csv

# Combine subjects and output trial-level

## Read in and concatenate subject datasheets

In [8]:
cleaned_dataframes_list = []

for s in os.listdir(source_dir):
    if s.startswith('sub-4'):
        sub_dir = os.path.join(source_dir,s)
        for f in os.listdir(sub_dir):
            if f.endswith('.csv') and f.split('_')[-2] == 'task-nback':
                fpath = os.path.join(sub_dir,f)
                df = pd.read_csv(fpath)
                cleaned_dataframes_list.append(df)
                print(df.head())

   sub  block  trial  CRsp  RT  Rsp  CR  MISS  FA  HIT
0  401      1      1     0 NaN    0   1     0   0    0
1  401      1      2     0 NaN    0   1     0   0    0
2  401      1      3     0 NaN    0   1     0   0    0
3  401      1      4     0 NaN    0   1     0   0    0
4  401      1      5     0 NaN    0   1     0   0    0
   sub  block  trial  CRsp  RT  Rsp  CR  MISS  FA  HIT
0  402      1      1     0 NaN    0   1     0   0    0
1  402      1      2     0 NaN    0   1     0   0    0
2  402      1      3     0 NaN    0   1     0   0    0
3  402      1      4     0 NaN    0   1     0   0    0
4  402      1      5     0 NaN    0   1     0   0    0
   sub  block  trial  CRsp  RT  Rsp  CR  MISS  FA  HIT
0  403      1      1     0 NaN    0   1     0   0    0
1  403      1      2     0 NaN    0   1     0   0    0
2  403      1      3     0 NaN    0   1     0   0    0
3  403      1      4     0 NaN    0   1     0   0    0
4  403      1      5     0 NaN    0   1     0   0    0
   sub  bl

In [10]:
nback_trial_level = pd.concat(cleaned_dataframes_list)

###### Output N-back trial-level data

In [11]:
nback_trial_level.to_csv(os.path.join(derivs_dir,'nback_trial_data.csv'),index=False)

### Group, expand, trim N-back data
Group by subjects, get the sum of all columns, the count of the trial column, and the mean of the RT column.

Establish Hit % `number of Hits / number of targets` and FA % `number of FAs / number of foils`. 

Corrected Recognition `HIT% - FA%`. 

In [12]:
grouped = nback_trial_level.groupby('sub')
nback_df = grouped.sum()
nback_df['trial'] = grouped.count()['trial']
nback_df['RT'] = grouped.mean()['RT']
nback_df['HIT%'] = nback_df['HIT'] / nback_df['CRsp']
nback_df['FA%'] = nback_df['FA'] / (nback_df['trial'] - nback_df['CRsp'])
nback_df['CoR'] = nback_df['HIT%'] - nback_df['FA%']
nback_df = nback_df[['RT','HIT%','FA%','CoR']]
nback_df.head()

,RT,HIT%,FA%,CoR
sub,,,,
401,642.000000,0.750,0.06250,0.68750
402,963.866667,0.750,0.28125,0.46875
403,946.444444,0.875,0.06250,0.81250
404,1466.400000,0.875,0.09375,0.78125
405,1000.875000,0.750,0.06250,0.68750


###### Output N-back subject-level data

In [13]:
nback_df.to_csv(os.path.join(derivs_dir,'nback_subject_data.csv'))